In [120]:
import pandas as pd
import pygsheets
import holidays
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [121]:
#Importation du token Asana
with open("token_asana.txt", "r") as file:
    asana_token = file.read().strip()

In [122]:
# ORDI FIXE Authentifier avec le fichier de clé JSON
#gc = pygsheets.authorize(service_file=r'D:\DATA\2024-11-2022_Tableau-Heures\feuille-heures-c0ab0678243d.json')

In [123]:
#PC PORTABLE
gc = pygsheets.authorize(service_file=r'D:\Documents\DATA\2024-11-24_CHEMDOC\2024-11-24_Feuille_Heures\Heure_chemdoc\feuille-heures-c0ab0678243d.json')

In [124]:
# Ouvrir la feuille Google Sheet par son titre
#spreadsheet = gc.open('2024-2026_TABLEAU DES HEURES')
spreadsheet = gc.open("RELEVE DES HEURES (REPONSES/NE PAS MODIFIER/FILTRER) ")

In [125]:
# Ouvrir la première feuille (DATA)
worksheet = spreadsheet.sheet1

In [126]:
#Importer toute les valeur et les convertir dans un df pandas
values = worksheet.get_all_values()
df = pd.DataFrame(values[1:], columns=values[0])


In [127]:
#Afficher toute les colonnes du df
pd.set_option('display.max_columns', None)

In [128]:
df = df.replace('', pd.NA)

In [129]:
#Supprimer lmes ligne sans valeur dans les colonnes Prénom Nom, Date
df = df.dropna(subset =["Prénom, NOM", "DATE"], how ="all").reset_index(drop =True)
print("Nombre de ligne conservées =",len(df))

Nombre de ligne conservées = 11491


In [130]:
#PREFILTRAGE DES DATE : Convertis la colone timestamp pour reduire la taille du df car la colonne date est trop hétérogène
df["Timestamp"] = pd.to_datetime(df["Timestamp"],format='mixed')

#Créer une colonne anné a partir de timestamp puis nous réduison la taille du DF a partir des date supérieur ou égale 2023
df["Annee_timestamp"] = df["Timestamp"].dt.year
df = df[df["Annee_timestamp"] >= 2023]
print("Nombre de ligne conservées =",len(df))

Nombre de ligne conservées = 8059


In [131]:
#Modifie les années abhérante observée dans date 0024 et 2004
df["DATE"] = df["DATE"].apply(lambda x : x.replace("0024", "2024").replace("2004", "2024"))

In [132]:
#Transforme la colonne date en datetime
df["DATE"] = pd.to_datetime(df["DATE"])

In [133]:
#Créé une colonne année, mois, n) de semaine a partir de la colonne DATE, les ligne 2022 sont peu donc conservées
df["Annee"] = df["DATE"].dt.year
df["Mois"] = df["DATE"].dt.strftime('%m-%B')
df["N_semaine"] = df["DATE"].dt.isocalendar().week

In [134]:
#identification de la list des intérims
list_interim = ['TECHNICIEN MONTEUR1', 'ELECTROTECHNICIEN INDUSTRIEL2', 'MANUTENTIONNAIRE INDUSTRIEL1','ELECTROTECHNICIEN INDUSTRIEL1']

#inverser le prénom et le nom en conservant la liste interim dans le même sens 
def invert_name(name) :
    if name in list_interim :
        return name
    else : 
        invert = ' '.join(name.split()[1:] + [name.split()[0]])
        return invert

df["Prénom, NOM"] = df["Prénom, NOM"].apply(invert_name)
#Renomage de la colonne en NOM_PRENOM
df.rename(columns ={"Prénom, NOM":"NOM_Prénom"}, inplace = True)

In [135]:
#Il y a plusieur colonne heure travaillé, nous renomons la première en HEURE_DECLAREE
#df.columns.values[5] = "HEURE_DECLAREE"

In [136]:
# Sélectionner uniquement les colonnes "Heures travaillées"
heures_travail = df.filter(like="Heures travaillées", axis=1)

#Transforme les virgules en point pour permettre la conversion en float
heures_travail = heures_travail.apply(lambda col: col.str.replace(",", "."))

# Convertir les colonnes en float
heures_travail = heures_travail.apply(pd.to_numeric)

# Agréger les colonnes "Heures travaillées" par addition, les collone source ne sont pas changer pour pouvoir visualiser les potentielle erreur de saisie
df["Heures_travaillées_total_jour"] = heures_travail.sum(axis=1)

In [137]:
#Transformation de la colone panier en binaire 1 = oui pour pouvoir la compter
df["Panier"] = df["Panier"].fillna(0)
df["Panier"] = df["Panier"].apply(lambda x : 1 if x =="oui" else 0)


In [138]:
#Compter les jours travailler si la valeur est différente de 1 = jours travaillé
df["JOUR_TRAVAILLEE"] = df["Heures_travaillées_total_jour"].apply(lambda x : 0 if x == 0 else 1)

In [197]:
# Fonction pour calculer les heures supplémentaires
def jours_ouvres(row):

    #Déclare les varibale correspondant à la ligne
    Annee = row["Annee"]
    Semaine = row["N_semaine"]
    Mois = row["DATE"].month

    #Défiie la date du lundi et du vendredi par rapport au numéro de semaine
    start_date = pd.Timestamp(f"{Annee}-01-01") + pd.Timedelta(weeks= Semaine - 1)
    start_date = start_date - pd.Timedelta(days=start_date.weekday())  # S'assurer que c'est un lundi
    end_date = start_date + pd.Timedelta(days=4)

    # Générer toutes les dates entre start_date et end_date
    all_dates = pd.date_range(start=start_date, end=end_date)
    # Filtrer pour ne conserver que les dates du mois donné,
    filtered_dates = []

    for date in all_dates :
        if date.month == Mois :
            filtered_dates.append(date)

    #Donne le nombre de jours dans la semaine en évitant les semaine coupés en deux
    jours_ouvres = len(filtered_dates)


    #Importer la list des jours fériés 
    fr_holidays = holidays.France(years=Annee)
    list_jours_feries = list(fr_holidays.keys())
    # Convertir les dates en Timestamp pour pouvoir les comparer directement
    for index, date in enumerate(list_jours_feries):
        list_jours_feries[index] = pd.Timestamp(date)

    # Trouver le nombre de jours férié
    found_jours_feries = []
    for date in filtered_dates :
        if date in list_jours_feries :
            found_jours_feries.append(date)

    jours_feries = len(found_jours_feries )

    #déduit les jours férié potentiel au nombre de jours ouvrable par semaine/mois 
    jours_ouvres = jours_ouvres - jours_feries
    
    # Retourner le maximum entre 0 et les heures supplémentaires (pas de valeurs négatives)
    return jours_ouvres


# Appliquer la fonction de calcul des heures supplémentaires pour chaque ligne du DataFrame
df['Jours_ouvres'] = df.apply(jours_ouvres, axis=1)


In [ ]:
#Calculer les heure de bases (prend en compte les jours ouvré et fériés, PAS LES CONGES)
df["Base_heure"] = df["Jours_ouvres"]*7

In [ ]:
#Calculer les heure supplémentaire (prend en compte les jours ouvré et fériés, PAS LES CONGES)
df["Heures_supp"]  = df.apply(lambda row: row['col1'] - row['col2'] if row['col1'] > row['co

In [200]:
df_test = df[(df["NOM_Prénom"] =="LIGOUT Lucien") & (df["Mois"] =="11-November") & (df["Annee"] ==2024)]
df_test

,Timestamp,NOM_Prénom,DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Score,Email Address,,,,,,,Annee_timestamp,Annee,Mois,N_semaine,Heures_travaillées_total_jour,JOUR_TRAVAILLEE,Jours_ouvres,Base_heure
10901,2024-11-05 21:02:54,LIGOUT Lucien,2024-11-05,CHANTIER,1,11,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,45,11.0,1,5,35
10937,2024-11-07 06:20:42,LIGOUT Lucien,2024-11-06,CHANTIER,1,9.5,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,45,9.5,1,5,35
10958,2024-11-07 19:30:27,LIGOUT Lucien,2024-11-07,CHANTIER,1,9.5,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,45,9.5,1,5,35
10988,2024-11-08 17:14:35,LIGOUT Lucien,2024-11-08,CHANTIER,1,8,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,45,8.0,1,5,35
11018,2024-11-12 19:01:41,LIGOUT Lucien,2024-11-12,CHANTIER,1,1,AUTRES - AUTRES,Oui,9.5,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,46,10.5,1,4,28
11053,2024-11-14 06:56:33,LIGOUT Lucien,2024-11-13,CHANTIER,1,9,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,46,9.0,1,4,28
11081,2024-11-14 20:03:06,LIGOUT Lucien,2024-11-14,CHANTIER,1,10,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,46,10.0,1,4,28
11094,2024-11-15 15:51:17,LIGOUT Lucien,2024-11-15,CHANTIER,1,6.5,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,46,6.5,1,4,28
11127,2024-11-18 17:03:20,LIGOUT Lucien,2024-11-18,BUREAU D'ETUDES,0,2,AUTRES - AUTRES,Oui,3,SUEZ - APERAM GUEUGNON - 23271210,Oui,2,ROASYS Reuse mixte/Eau Industrielle - I24277113,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47,7.0,1,5,35
11160,2024-11-20 08:43:04,LIGOUT Lucien,2024-11-19,BUREAU D'ETUDES,0,3,AUTRES - AUTRES,Oui,4,SUEZ - APERAM GUEUGNON - 23271210,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47,7.0,1,5,35


In [159]:
df

,Timestamp,NOM_Prénom,DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,Score,Email Address,,,,,,,Annee_timestamp,Annee,Mois,N_semaine,Heures_travaillées_total_jour,JOUR_TRAVAILLEE,Jours_ouvres
1,2024-10-25 00:00:00,EL KANDOUSSI Mehdi,2024-10-18,ATELIER,0,7,GRAND LAC - REUT - 24222172,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,mehdi.el-kandoussi@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,10-October,42,7.0,1,5
2,2024-03-05 00:00:00,PIQUET François,2024-11-20,BUREAU D'ETUDES,0,7,ISCHIA-KORDIANCE - 22272277,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,piquetfrosty@gmail.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47,7.0,1,5
3,2024-05-27 00:00:00,PIQUET François,2024-11-20,ATELIER,0,8,DS SMITH - 23201341,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,piquetfrosty@gmail.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47,8.0,1,5
4,2024-03-06 00:00:00,PIQUET François,2024-11-20,ATELIER,0,8,DS SMITH - 23201341,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,piquetfrosty@gmail.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47,8.0,1,5
1243,2023-01-27 00:00:00,LIGOUT Lucien,2022-01-13,BUREAU D'ETUDES,0,7.00,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,lucien.ligout@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2023,2022,01-January,2,7.0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11486,2024-12-05 09:47:11,RAMOND-ZWILLER Atalande,2024-12-04,BUREAU D'ETUDES,0,8,AUTRES - AUTRES,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,atalande.ramond-zwiller@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,12-December,49,8.0,1,5
11487,2024-12-05 12:42:16,DISPA Cécile,2024-12-02,BUREAU D'ETUDES,0,8,AUTRES - AUTRES,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,cecile.dispa@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,12-December,49,8.0,1,5
11488,2024-12-05 12:42:43,DISPA Cécile,2024-12-03,BUREAU D'ETUDES,0,8,AUTRES - AUTRES,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,cecile.dispa@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,12-December,49,8.0,1,5
11489,2024-12-05 12:43:15,DISPA Cécile,2024-12-04,BUREAU D'ETUDES,0,8,AUTRES - AUTRES,Non,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,cecile.dispa@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,12-December,49,8.0,1,5


In [141]:
#exportation des données dans google sheet
#spreadsheet_export = gc.open('2024-2026_TABLEAU DES HEURES')
#Choisis lapremière feuille
#worksheet = spreadsheet_export[0]

#selectionnne la cellule d'arrivé
#worksheet.set_dataframe(df_agg, (1, 1))